# cost_calculations

> File **cost_calculation.py** will contain all of the cost calculation functions

In [ ]:
#| default_exp cost_calculations

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

#### Core Python Data Analysis,
import pandas as pd 
import numpy as np
import janitor as jn
import plotly.express as px
import pandas_flavor as pf
import os
import sqlalchemy as sql

In [ ]:
#| export

#### Function: Calculate Monthly Unsubscriber Cost Table

def cost_calc_monthly_cost_table(
    email_list_size = 1e5,
    email_list_growth_rate = 0.035,
    sales_emails_per_month = 5,
    unsub_rate_per_sales_email = 0.005,
    customer_conversion_rate = 0.05,
    average_customer_value = 2000,
    n_periods = 12
    ):
    """This function generates a Cost Table.
        Args:
            email_list_size (_type_, optional): Email list size. Defaults to 1e5.
            email_list_growth_rate (float, optional): Monthly email list growth rate. Defaults to 0.035.
            sales_emails_per_month (int, optional): Sales email per month. Defaults to 5.
            unsub_rate_per_sales_email (float, optional): Unsubscription rate per email. Defaults to 0.005.
            customer_conversion_rate (float, optional): Rate of email subscribers that convert to customers. Defaults to 0.05.
            average_customer_value (int, optional): Average customer value. Defaults to 2000.
            n_periods (int, optional): Number of months for cost table. Defaults to 12.
    
        Returns:
            DataFrame: Returns a Cost Table
    """

    # Period
    period_series = pd.Series(np.arange(0, n_periods), name = "period")
    
    cost_table_df = period_series.to_frame()
    
    # Email Size - No Growth
    # np.repeat(): Repeates a value to a user-defined number of times
    cost_table_df['email_size_no_growth'] = np.repeat(email_list_size, n_periods)
    
    # Lost Customers - No Growth
    cost_table_df['lost_customers_no_growth'] = cost_table_df['email_size_no_growth'] * unsub_rate_per_sales_email * sales_emails_per_month * customer_conversion_rate
    
    # Cost - No Growth
    cost_table_df['cost_no_growth'] = cost_table_df['lost_customers_no_growth'] * average_customer_value

    # Email Size - With Growth
    cost_table_df['email_size_with_growth'] = cost_table_df['email_size_no_growth'] * (( 1 + email_list_growth_rate) ** cost_table_df['period'])
    
    # Lost Customers - With Growth
    cost_table_df['lost_customers_with_growth'] = cost_table_df['email_size_with_growth'] * unsub_rate_per_sales_email * sales_emails_per_month * customer_conversion_rate
    
    # Cost - With Growth
    cost_table_df['cost_with_growth'] = cost_table_df['lost_customers_with_growth'] * average_customer_value
    
    return cost_table_df

In [ ]:
#| export

#### Summary Function: You'll need to report a single value for the lost revenue.
### @pf.register_dataframe_method # Using pandas_flavor package: @pf.register_dataframe_method** Turns a function into a data frame method so we can use method chaining

@pf.register_dataframe_method
def cost_total_unsub_cost(cost_table):
    """Takes the input from cost_calc_monthly_cost_table(), and produces a summary of the total costs.
        Args:
            cost_table (DataFrame): Output fro cost_calc_monthly_cost_table()
    
        Returns:
            DataFrame: Summarized total costs for email unsubscription.
    """
    summary_df = cost_table[['cost_no_growth', 'cost_with_growth']].sum().to_frame().transpose()
    
    return summary_df

In [ ]:
#| export

### Function: Simulate Unsubscriber Costs

def cost_simulate_unsub_costs(
    email_list_monthly_growth_rate = [0, 0.035],
    customer_conversion_rate = [0.04, 0.05, 0.06],
    **kwargs):
    """Generate a cost analysis simulation to characterize cost uncertainty.

    Args:
        email_list_monthly_growth_rate (list, optional): List of values for email monthly growth rate to simulate. Defaults to [0, 0.035].
        customer_conversion_rate (list, optional): List of values for customer conversion rate to simulate. Defaults to [0.04, 0.05, 0.06].
    
    Returns:
        DataFrame: Cartesian Product of email list and customer conversion rate is calculated and total unsubscriber costs are calculated.
    """
    
    # Parameter Grid    
    data_dict = dict(
    email_list_monthly_growth_rate = email_list_monthly_growth_rate,
    customer_conversion_rate = customer_conversion_rate
    )

# Add `.droplevel(axis=1,level=1)` to the jn.expand_grid(others = data_dict) in order to drop the extra letter ' 0' added to the column names.
# Apparently: It was not an issue with pyjanitor 0.21.2. But it becomes an issue when using using a newer version pyjnaitor 0.24.0 or after.
    parameter_grid_df = jn.expand_grid(others = data_dict).droplevel(axis=1,level=1)
    
    # Temporary Function
    def temporary_function(x, y):
            
        cost_table_df = cost_calc_monthly_cost_table(
            email_list_growth_rate = x,
            customer_conversion_rate = y,
            **kwargs
            )
    
        summary_df = cost_total_unsub_cost(cost_table_df)
        
        return summary_df
    
    # List Comprehension
    summary_list = [temporary_function(x, y) for x, y in zip(parameter_grid_df['email_list_monthly_growth_rate'], parameter_grid_df['customer_conversion_rate'])]

    summary_results_df = pd.concat(summary_list, axis = 0)         .reset_index()         .drop('index', axis = 1)         .merge(parameter_grid_df, left_index = True, right_index = True)

    return summary_results_df

In [ ]:
#| export

### Function: Plot Simulation

@pf.register_dataframe_method
def cost_plot_simulated_unsub_costs(simulation_results):
    """A plottign function to plot the results from cost_simulate_unsub_costs() function. 

    Args:
        simulation_results (DataFrame): The output from cost_simulate_unsub_costs()

    Returns:
        Plotly Plot: Heatmap that visualizes the cost simulation.
    """
    
    simulation_results_wide_df = simulation_results     .drop('cost_no_growth', axis = 1)     .pivot(
        index = 'email_list_monthly_growth_rate',
        columns = 'customer_conversion_rate',
        values = 'cost_with_growth'
        )
    
    fig = px.imshow(
    simulation_results_wide_df,
    origin = 'lower',
    aspect = 'auto',
    title = 'Lead Cost Simulation',
    labels = dict(
        x = 'Customer Conversion Rate',
        y = 'Monthly Email Growth Rate',
        color = 'Cost of Unsubscription')
        )
        
    print(simulation_results_wide_df)
    
    return fig


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()